In [115]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_regression, SelectKBest
import xgboost as xgb
from collections import Counter
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,mean_absolute_percentage_error
from statsmodels.stats.outliers_influence import variance_inflation_factor 
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

In [116]:

df = pd.read_csv("../data/datasets/raw/0_VERSION_FINAL_BLANCOS.csv")

In [117]:
pd.set_option("display.max_colwidth", None)

In [118]:
df["Precio"] = df["Precio"].fillna("").astype(str).str.replace(",", "", regex=True)


In [119]:
df["Precio"] = pd.to_numeric(df["Precio"], errors="coerce")

In [120]:
df["Valoración"] = pd.to_numeric(df["Valoración"], errors="coerce")

In [121]:
def limpiar_caza(texto):
    if isinstance(texto, str) and "caza" in texto.lower():  # Verifica si es texto y contiene "caza"
        return pd.Series(texto).replace(r"\s*\(.*?\)", "", regex=True).iloc[0]
    return texto  # Devuelve el texto sin cambios si no tiene "caza"

# Aplicar la limpieza a todas las celdas del DataFrame
df = df.applymap(limpiar_caza)

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_8394/3888707711.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(limpiar_caza)


In [122]:
# Modificamos maridaje porque contiene una "," en medio de la descrpción 
def limpiar_pesca(texto):
    if isinstance(texto, str) and "azul " in texto.lower():  # Verifica si es texto y contiene "caza"
        return pd.Series(texto).replace(r"\s*\(.*?\)", "", regex=True).iloc[0]
    return texto  # Devuelve el texto sin cambios si no tiene "caza"

# Aplicar la limpieza a todas las celdas del DataFrame
df = df.applymap(limpiar_pesca)

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_8394/979100320.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(limpiar_pesca)


In [123]:
df["Maridajes"]

0                                                Marisco, Vegetariana, Aperitivos y tentempiés
1                   Cerdo, Marisco, Aves, Pescado azul, Pescado blanco, Queso tierno y cremoso
2                                                                                No disponible
3                   Vegetariana, Pasta, Aperitivos y tentempiés, Pescado blanco, Carne adobada
4                          Marisco, Pasta, Pescado azul, Queso tierno y cremoso, Carne adobada
                                                 ...                                          
11656                              Marisco, Pasta, Queso curado, Pescado blanco, Carne adobada
11657    Marisco, Aves, Comida picante, Pescado blanco, Queso de leche de cabra, Carne adobada
11658                              Marisco, Aperitivos y tentempiés, Pescado blanco, Aperitivo
11659                                        Cerdo, Comida picante, Champiñones, Carne adobada
11660                                       Marisc

In [124]:
#Separamos la columna maridajes para que cada una sea una característica
df_maridaje_separado =  df["Maridajes"].str.split(',', expand=True)
df_maridaje_separado["ID"]= df["ID"]
df_maridaje_separado = df_maridaje_separado.fillna("Desconocido")
df_maridaje_separado[100:140]

,0,1,2,3,4,5,ID
100,Cerdo,Vegetariana,Aves,Aperitivos y tentempiés,Pescado blanco,Desconocido,6771529
101,Marisco,Vegetariana,Comida picante,Aperitivos y tentempiés,Pescado blanco,Desconocido,21883
102,Marisco,Vegetariana,Aves,Pescado blanco,Desconocido,Desconocido,8037379
103,No disponible,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,1680097
104,Marisco,Pescado azul,Comida picante,Aperitivos y tentempiés,Pescado blanco,Carne adobada,5664291
105,Cerdo,Vegetariana,Pescado azul,Comida picante,Carne adobada,Desconocido,4774645
106,Marisco,Pescado azul,Queso tierno y cremoso,Carne adobada,Desconocido,Desconocido,2489211
107,Marisco,Aves,Comida picante,Pescado blanco,Queso de leche de cabra,Carne adobada,47154
108,No disponible,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,5341468
109,Marisco,Pasta,Queso curado,Pescado blanco,Carne adobada,Desconocido,4722357


In [125]:
valores_unicos_maridaje = set(df_maridaje_separado.drop(columns=["ID"]).values.ravel())
print(valores_unicos_maridaje)

{'Cordero', 'Marisco', 'No disponible', ' Aperitivos y tentempiés', 'Desconocido', 'Ternera lechal', 'Aves', ' Pescado blanco', 'Aperitivos y tentempiés', ' Pescado azul', ' Aperitivo', 'Cerdo', ' Cerdo', ' Marisco', ' Comida picante', ' La comida basura vale', ' Queso curado', 'Vegetariana', ' Champiñones', ' Aves', ' Queso de leche de cabra', ' Carne adobada', ' Vegetariana', ' Queso tierno y cremoso', ' Pasta'}


In [126]:
df_maridaje_separado_sin_ID = df_maridaje_separado.drop(columns = "ID")

In [127]:
df_maridaje_separado_sin_ID = pd.get_dummies(df_maridaje_separado_sin_ID[[0,1]], prefix='', prefix_sep='')

In [128]:
df_maridaje_separado_sin_ID

,Aperitivos y tentempiés,Aves,Cerdo,Cordero,Marisco,No disponible,Ternera lechal,Vegetariana,Aperitivo,Aperitivos y tentempiés,...,Cerdo,Comida picante,La comida basura vale,Marisco,Pasta,Pescado azul,Pescado blanco,Queso curado,Vegetariana,Desconocido
0,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,False,True,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False
4,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
11657,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11658,False,False,False,False,True,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
11659,False,False,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [129]:
df_maridaje_separado_sin_ID = df_maridaje_separado_sin_ID.drop(columns = ["Desconocido","No disponible"])

In [130]:
df_maridaje_separado_sin_ID = df_maridaje_separado_sin_ID.applymap(lambda x: x.replace(" ", "") if isinstance(x, str) else x)
# Convertir todo el DataFrame a One-Hot Encoding sin prefijos por columna
df_maridaje_separado_sin_ID_encoded = pd.get_dummies(df_maridaje_separado_sin_ID,drop_first=True ,prefix="" ,prefix_sep="")

# Convertir a DataFrame
df_maridaje_separado_sin_ID_encoded = pd.DataFrame(df_maridaje_separado_sin_ID_encoded)

# Mostrar resultado
display(df_maridaje_separado_sin_ID_encoded)

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_8394/2076073784.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_maridaje_separado_sin_ID = df_maridaje_separado_sin_ID.applymap(lambda x: x.replace(" ", "") if isinstance(x, str) else x)


,Aperitivos y tentempiés,Aves,Cerdo,Cordero,Marisco,Ternera lechal,Vegetariana,Aperitivo,Aperitivos y tentempiés,Aves,Cerdo,Comida picante,La comida basura vale,Marisco,Pasta,Pescado azul,Pescado blanco,Queso curado,Vegetariana
0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False
11657,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False
11658,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
11659,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False


In [131]:
df_maridaje_separado_sin_ID_encoded.columns = df_maridaje_separado_sin_ID_encoded.columns.str.strip()

In [132]:
df_maridaje_separado_sin_ID_encoded=df_maridaje_separado_sin_ID_encoded.groupby(axis=1, level=0).sum()
df_maridaje_separado_encoded = df_maridaje_separado_sin_ID_encoded
df_maridaje_separado_encoded["ID"] = df["ID"]
df_maridaje_separado_encoded

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_8394/1026340629.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_maridaje_separado_sin_ID_encoded=df_maridaje_separado_sin_ID_encoded.groupby(axis=1, level=0).sum()


,Aperitivo,Aperitivos y tentempiés,Aves,Cerdo,Comida picante,Cordero,La comida basura vale,Marisco,Pasta,Pescado azul,Pescado blanco,Queso curado,Ternera lechal,Vegetariana,ID
0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,15819
1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1514997
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4262558
3,0,0,0,0,0,0,0,0,1,0,0,0,0,1,5707221
4,0,0,0,0,0,0,0,1,1,0,0,0,0,0,7009177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1736573
11657,0,0,1,0,0,0,0,1,0,0,0,0,0,0,3125229
11658,0,1,0,0,0,0,0,1,0,0,0,0,0,0,2219168
11659,0,0,0,1,1,0,0,0,0,0,0,0,0,0,5233952


In [133]:
df


,Url,ID,Nombre,Año,País,Región,Bodega,Tipo de vino,Uva,Precio,...,Débil/Ácido,Taste_Notes_1_1,Taste_Notes_1_2,Taste_Notes_1_3,Taste_Notes_2_1,Taste_Notes_2_2,Taste_Notes_2_3,Taste_Notes_3_1,Taste_Notes_3_2,Taste_Notes_3_3
0,https://www.vivino.com/ES/es/sella-mosca-monteoro-vermentino-di-gallura-superiore/w/15819,15819,Monteoro Vermentino di Gallura Superiore,2023,Italia,Vermentino di Gallura,Sella & Mosca,Vino blanco,Vermentino,16.38,...,3.4,pomelo,menta,piedra,cítrico,limón,pomelo,minerales,minerales,piedra
1,https://www.vivino.com/ES/es/triebaumer-gelber-muskateller/w/1514997,1514997,Gelber Muskateller,No disponible,Austria,Neusiedlersee-Hügelland,Triebaumer,Vino blanco,Gelber Muskateller,13.69,...,8.0,menta,pimienta verde,mora,limón,limón,banana,algodón de azúcar,ajedrea,polvo de vainilla
2,https://www.vivino.com/ES/es/vegalfaro-macabeo-chardonnay/w/4262558,4262558,Caprasia Blanco,2021,España,Valencia,Vegalfaro,Vino blanco,"Chardonnay, Macabeo",12.00,...,4.1,limón,pimienta de Jamaica,cítrico,piedra,menta,drupa,roble,mantequilla,polvo de vainilla
3,https://www.vivino.com/ES/es/adanti-montefalco-grechetto-montefalco/w/5707221,5707221,Montefalco Grechetto,2023,Italia,Montefalco,Adanti,Vino blanco,Grechetto,13.84,...,5.1,pomelo,menta,piedra,minerales,minerales,piedra,cítrico,polvo de vainilla,polvo de vainilla
4,https://www.vivino.com/ES/es/rochebin-clos-saint-germain-bourgogne-chardonnay/w/7009177,7009177,Clos Saint Germain Bourgogne Chardonnay,No disponible,Francia,Bourgogne,Domaine de Rochebin,Vino blanco,Chardonnay,12.90,...,3.4,roble,mantequilla,uva,limón,limón,cítrico,menta,pomelo,piedra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,https://www.vivino.com/ES/es/corte-viola-bianco-di-custoza/w/1736573,1736573,Bianco di Custoza,2023,Italia,Bianco di Custoza,Corte Viola,Vino blanco,"Chardonnay, Pinot Blanc, Cortese, Garganega",7.98,...,No disponible,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla
11657,https://www.vivino.com/ES/es/domaine-saint-lannes-sauvignon-chardonnay/w/3125229,3125229,Sauvignon - Chardonnay,No disponible,Francia,Côtes de Gascogne,Domaine Saint-Lannes,Vino blanco,"Chardonnay, Sauvignon Blanc",6.40,...,No disponible,cítrico,limón,pomelo,ciruela pasa,polvo de vainilla,polvo de vainilla,pomelo,polvo de vainilla,polvo de vainilla
11658,https://www.vivino.com/ES/es/vinos-atxega-kurmisti-blanco/w/2219168,2219168,Kurmisti Blanco,No disponible,España,País Vasco,Vinos Atxega,Vino blanco,Hondarrabi Zuri,5.00,...,No disponible,pimienta verde,menta,No disponible,cítrico,polvo de vainilla,polvo de vainilla,hierbas,polvo de vainilla,polvo de vainilla
11659,https://www.vivino.com/ES/es/bergdolt-reif-nett-tradition-grauburgunder-trocken/w/5233952,5233952,Tradition Grauburgunder Trocken,No disponible,Alemania,Pfalz,Bergdolt-Reif & Nett,Vino blanco,Pinot Grigio,8.50,...,No disponible,cítrico,limón,limón,minerales,polvo de vainilla,polvo de vainilla,menta,polvo de vainilla,polvo de vainilla


In [134]:
df_uvas_separado =  df["Uva"].str.split(',', expand=True)
df_uvas_separado["ID"]= df["ID"]
df_uvas_separado = df_uvas_separado.fillna("Desconocido")
df_uvas_separado[100:140]

,0,1,2,3,4,5,6,7,8,9,10,11,12,ID
100,Sauvignon Blanc,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,6771529
101,Müller-Thurgau,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,21883
102,Sauvignon Blanc,Mauzac Blanc,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,8037379
103,Grenache Blanc,Macabeo,Chardonnay,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,1680097
104,Roussanne,Rolle,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,5664291
105,Riesling,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,4774645
106,Weissburgunder,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,2489211
107,Gros Manseng,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,47154
108,Malagouzia,Assyrtiko,Rhoditis,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,5341468
109,Sauvignon Blanc,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,4722357


In [135]:
df_uvas_separado = df_uvas_separado.drop(columns = "ID")

df_uvas_separado = df_uvas_separado.applymap(lambda x: x.replace(" ", "") if isinstance(x, str) else x)
# Convertir todo el DataFrame a One-Hot Encoding sin prefijos por columna
df_uvas_separado = pd.get_dummies(df_uvas_separado,drop_first=True ,prefix="" ,prefix_sep="")

# Convertir a DataFrame
df_uvas_separado = pd.DataFrame(df_uvas_separado)

# Mostrar resultado
display(df_uvas_separado)

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_8394/3747642572.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_uvas_separado = df_uvas_separado.applymap(lambda x: x.replace(" ", "") if isinstance(x, str) else x)


,Aidani,Airen,Alarije,Albana,AlbarinBlanco,Albariño,Albarola,Albillo,AlbilloMayor,AlicanteBouschet,...,GrünerVeltliner,Müller-Thurgau,Traminer,PinotBeurot,PinotBlanc,Rotgipfler,GrünerVeltliner,Welschriesling,Riesling,Scheurebe
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11657,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11658,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11659,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [136]:
df_uvas_separado = df_uvas_separado.drop(columns="Desconocido")
df_uvas_separado=df_uvas_separado.groupby(axis=1, level=0).sum()
df_uvas_separado

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_8394/1971516612.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_uvas_separado=df_uvas_separado.groupby(axis=1, level=0).sum()


,Aidani,Airen,Alarije,Albana,Albanella,AlbarinBlanco,Albariño,Albarola,Albillo,AlbilloMayor,...,Weissburgunder,Welschriesling,Xarel-lo,Xinomavro,Zakynthino,Zalema,Zelen,Zengö,Zibibbo,Zierfandler
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11657,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11658,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11659,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [137]:
df_uvas_separado.dtypes

Aidani         int64
Airen          int64
Alarije        int64
Albana         int64
Albanella      int64
               ...  
Zalema         int64
Zelen          int64
Zengö          int64
Zibibbo        int64
Zierfandler    int64
Length: 450, dtype: object

In [138]:
# Definir el umbral
umbral = 100

# Filtrar columnas cuya suma sea mayor al umbral
columnas_filtradas = df_uvas_separado.loc[:, df_uvas_separado.sum() > umbral]

columnas_filtradas

,Albariño,Chardonnay,CheninBlanc,Garganega,GarnachaBlanca,Gewürztraminer,Godello,GrenacheBlanc,GrünerVeltliner,Macabeo,...,Riesling,Roussanne,SauvignonBlanc,Sémillon,Verdejo,Vermentino,Viognier,Viura,Weissburgunder,Xarel-lo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11657,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
11658,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11659,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [139]:
df_uvas_separado_filtro = columnas_filtradas

In [140]:
df_uvas_separado_filtro["ID"] = df["ID"]

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_8394/3015606806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uvas_separado_filtro["ID"] = df["ID"]


In [141]:
df_notas_sabor_mejorardo = df[["Taste_Notes_1_1","Taste_Notes_1_2","Taste_Notes_1_3","Taste_Notes_2_1","Taste_Notes_2_2","Taste_Notes_2_3","Taste_Notes_3_1","Taste_Notes_3_2","Taste_Notes_3_3","ID"]]


In [142]:
df_notas_sabor_mejorardo

,Taste_Notes_1_1,Taste_Notes_1_2,Taste_Notes_1_3,Taste_Notes_2_1,Taste_Notes_2_2,Taste_Notes_2_3,Taste_Notes_3_1,Taste_Notes_3_2,Taste_Notes_3_3,ID
0,pomelo,menta,piedra,cítrico,limón,pomelo,minerales,minerales,piedra,15819
1,menta,pimienta verde,mora,limón,limón,banana,algodón de azúcar,ajedrea,polvo de vainilla,1514997
2,limón,pimienta de Jamaica,cítrico,piedra,menta,drupa,roble,mantequilla,polvo de vainilla,4262558
3,pomelo,menta,piedra,minerales,minerales,piedra,cítrico,polvo de vainilla,polvo de vainilla,5707221
4,roble,mantequilla,uva,limón,limón,cítrico,menta,pomelo,piedra,7009177
...,...,...,...,...,...,...,...,...,...,...
11656,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla,1736573
11657,cítrico,limón,pomelo,ciruela pasa,polvo de vainilla,polvo de vainilla,pomelo,polvo de vainilla,polvo de vainilla,3125229
11658,pimienta verde,menta,No disponible,cítrico,polvo de vainilla,polvo de vainilla,hierbas,polvo de vainilla,polvo de vainilla,2219168
11659,cítrico,limón,limón,minerales,polvo de vainilla,polvo de vainilla,menta,polvo de vainilla,polvo de vainilla,5233952


In [143]:
df_taste_1_separado_tijera_sin_ID = df_notas_sabor_mejorardo.drop(columns="ID")

In [144]:
valores_unicos_taste = set(df_taste_1_separado_tijera_sin_ID.values.ravel())

# Filtrar None y aplicar strip() solo a los strings
valores_unicos_taste = {s.strip() for s in valores_unicos_taste if isinstance(s, str)}

print(valores_unicos_taste)

{'acelga', 'jalapeño', 'fruta negra', 'arándano', 'cedro', '', 'cuero', 'ceniza', 'mermelada de mora', 'higo seco', 'zarzamora', 'carne a la parrilla', 'almendra', 'fresa madura', 'fresa', 'nuez moscada', 'queso', 'mantequilla', 'tofi', 'trufa', 'alquitrán', 'ciruela agria', 'bebida cola', 'cigarrera', 'pimienta de Jamaica', 'pasas de uva negras', 'especias exóticas', 'cítrico', 'crema', 'mermelada de bayas', 'sándalo', 'guinda', 'mezcla de especias para vino caliente', 'arándano rojo deshidratado', 'romero', 'uva pasa', 'mulberry', 'mora', 'carne adobada', 'carne de caza', 'suelo del bosque', 'salsa de fresa', 'pomelo', 'hierro', 'peonía', 'chocolate amargo', 'bergamota', 'virutas', 'No disponible', 'pétalo de rosa', 'algodón de azúcar', 'casis', 'menta', 'tiza', 'nuez negra', 'pizarra', 'grafito', 'hongo', 'caucho', 'eucalipto', 'canela', 'granada', 'anís', 'fruta seca', 'cereza roja', 'hierbas', 'arándano rojo', 'uva', 'cera de abejas', 'hibisco', 'solución salina', 'pimienta blanca

In [145]:
# Convertir todo el DataFrame a One-Hot Encoding sin prefijos por columna
df_taste_1_one_hot_encoded = pd.get_dummies(df_taste_1_separado_tijera_sin_ID, prefix='', prefix_sep='')

# Convertir a DataFrame
df_taste_1_one_hot_encoded = pd.DataFrame(df_taste_1_one_hot_encoded)

df_taste_1_one_hot_encoded.columns= df_taste_1_one_hot_encoded.columns.str.strip()
# Mostrar resultado
display(df_taste_1_one_hot_encoded)

,aceite balsámico,acelga,ajedrea,algodón de azúcar,almendra,almizcle,anís,arándano,arándano rojo,banana,...,tofi,tomate,tomillo,tostada,trufa,trufa negra,uva,uva pasa,vainilla,virutas
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11657,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11658,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11659,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [146]:
df_taste_1_one_hot_encoded=df_taste_1_one_hot_encoded.groupby(axis=1, level=0).sum()
df_taste_1_one_hot_encoded

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_8394/770153049.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_taste_1_one_hot_encoded=df_taste_1_one_hot_encoded.groupby(axis=1, level=0).sum()


,,No disponible,T,aceite balsámico,acelga,ajedrea,algodón de azúcar,almendra,almizcle,alquitrán,...,tomate,tomillo,tostada,trufa,trufa negra,uva,uva pasa,vainilla,virutas,zarzamora
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11657,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11658,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11659,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [147]:
# Definir el umbral
umbral = 250

# Filtrar columnas cuya suma sea mayor al umbral
columnas_filtradas = df_taste_1_one_hot_encoded.loc[:, df_taste_1_one_hot_encoded.sum() > umbral]

columnas_filtradas

df_taste_1_one_hot_encoded = columnas_filtradas

In [148]:
df_taste_1_one_hot_encoded = df_taste_1_one_hot_encoded.applymap(lambda x: 1 if x != 0 else 0)

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_8394/1271349127.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_taste_1_one_hot_encoded = df_taste_1_one_hot_encoded.applymap(lambda x: 1 if x != 0 else 0)


In [149]:
df_taste_1_one_hot_encoded = df_taste_1_one_hot_encoded.drop(columns="No disponible")

In [150]:
conteo_valores_unicos_taste = df_taste_1_one_hot_encoded.sum()

In [151]:
conteo_valores_unicos_taste

almizcle              344
banana                696
crema                 384
cítrico              5776
drupa                 405
grosella roja         303
hierro                272
hongo                 496
humo                  400
limón                5318
mantequilla          1642
menta                4401
minerales            5031
mora                  940
naranja               608
piedra               5153
pimienta verde       1978
pizarra               296
polvo de vainilla    7143
pomelo               5158
queso                 267
roble                1724
sal                   664
solución salina       347
sílex                 399
tabaco                419
tabaco dulce         1435
terroso               473
tiza                  286
tomillo              1161
uva                   869
vainilla              653
dtype: int64

In [152]:
df_taste_1_one_hot_encoded["ID"] = df["ID"]

In [153]:
df_merged_one_hot = pd.concat([df.set_index("ID"), df_maridaje_separado_encoded.set_index("ID"), df_taste_1_one_hot_encoded.set_index("ID"),df_uvas_separado_filtro.set_index("ID")], axis=1).reset_index()

display(df_merged_one_hot.head())

,ID,Url,Nombre,Año,País,Región,Bodega,Tipo de vino,Uva,Precio,...,Riesling,Roussanne,SauvignonBlanc,Sémillon,Verdejo,Vermentino,Viognier,Viura,Weissburgunder,Xarel-lo
0,15819,https://www.vivino.com/ES/es/sella-mosca-monteoro-vermentino-di-gallura-superiore/w/15819,Monteoro Vermentino di Gallura Superiore,2023,Italia,Vermentino di Gallura,Sella & Mosca,Vino blanco,Vermentino,16.38,...,0,0,0,0,0,1,0,0,0,0
1,1514997,https://www.vivino.com/ES/es/triebaumer-gelber-muskateller/w/1514997,Gelber Muskateller,No disponible,Austria,Neusiedlersee-Hügelland,Triebaumer,Vino blanco,Gelber Muskateller,13.69,...,0,0,0,0,0,0,0,0,0,0
2,4262558,https://www.vivino.com/ES/es/vegalfaro-macabeo-chardonnay/w/4262558,Caprasia Blanco,2021,España,Valencia,Vegalfaro,Vino blanco,"Chardonnay, Macabeo",12.00,...,0,0,0,0,0,0,0,0,0,0
3,5707221,https://www.vivino.com/ES/es/adanti-montefalco-grechetto-montefalco/w/5707221,Montefalco Grechetto,2023,Italia,Montefalco,Adanti,Vino blanco,Grechetto,13.84,...,0,0,0,0,0,0,0,0,0,0
4,7009177,https://www.vivino.com/ES/es/rochebin-clos-saint-germain-bourgogne-chardonnay/w/7009177,Clos Saint Germain Bourgogne Chardonnay,No disponible,Francia,Bourgogne,Domaine de Rochebin,Vino blanco,Chardonnay,12.90,...,0,0,0,0,0,0,0,0,0,0


In [154]:

df_merged_one_hot = df_merged_one_hot.dropna(axis=0, how="any")

In [155]:
df_merged_one_hot

,ID,Url,Nombre,Año,País,Región,Bodega,Tipo de vino,Uva,Precio,...,Riesling,Roussanne,SauvignonBlanc,Sémillon,Verdejo,Vermentino,Viognier,Viura,Weissburgunder,Xarel-lo
0,15819,https://www.vivino.com/ES/es/sella-mosca-monteoro-vermentino-di-gallura-superiore/w/15819,Monteoro Vermentino di Gallura Superiore,2023,Italia,Vermentino di Gallura,Sella & Mosca,Vino blanco,Vermentino,16.38,...,0,0,0,0,0,1,0,0,0,0
1,1514997,https://www.vivino.com/ES/es/triebaumer-gelber-muskateller/w/1514997,Gelber Muskateller,No disponible,Austria,Neusiedlersee-Hügelland,Triebaumer,Vino blanco,Gelber Muskateller,13.69,...,0,0,0,0,0,0,0,0,0,0
2,4262558,https://www.vivino.com/ES/es/vegalfaro-macabeo-chardonnay/w/4262558,Caprasia Blanco,2021,España,Valencia,Vegalfaro,Vino blanco,"Chardonnay, Macabeo",12.00,...,0,0,0,0,0,0,0,0,0,0
3,5707221,https://www.vivino.com/ES/es/adanti-montefalco-grechetto-montefalco/w/5707221,Montefalco Grechetto,2023,Italia,Montefalco,Adanti,Vino blanco,Grechetto,13.84,...,0,0,0,0,0,0,0,0,0,0
4,7009177,https://www.vivino.com/ES/es/rochebin-clos-saint-germain-bourgogne-chardonnay/w/7009177,Clos Saint Germain Bourgogne Chardonnay,No disponible,Francia,Bourgogne,Domaine de Rochebin,Vino blanco,Chardonnay,12.90,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,1736573,https://www.vivino.com/ES/es/corte-viola-bianco-di-custoza/w/1736573,Bianco di Custoza,2023,Italia,Bianco di Custoza,Corte Viola,Vino blanco,"Chardonnay, Pinot Blanc, Cortese, Garganega",7.98,...,0,0,0,0,0,0,0,0,0,0
11657,3125229,https://www.vivino.com/ES/es/domaine-saint-lannes-sauvignon-chardonnay/w/3125229,Sauvignon - Chardonnay,No disponible,Francia,Côtes de Gascogne,Domaine Saint-Lannes,Vino blanco,"Chardonnay, Sauvignon Blanc",6.40,...,0,0,1,0,0,0,0,0,0,0
11658,2219168,https://www.vivino.com/ES/es/vinos-atxega-kurmisti-blanco/w/2219168,Kurmisti Blanco,No disponible,España,País Vasco,Vinos Atxega,Vino blanco,Hondarrabi Zuri,5.00,...,0,0,0,0,0,0,0,0,0,0
11659,5233952,https://www.vivino.com/ES/es/bergdolt-reif-nett-tradition-grauburgunder-trocken/w/5233952,Tradition Grauburgunder Trocken,No disponible,Alemania,Pfalz,Bergdolt-Reif & Nett,Vino blanco,Pinot Grigio,8.50,...,0,0,0,0,0,0,0,0,0,0


In [156]:
df_merged_one_hot["ID"] = df_merged_one_hot["ID"].astype(int)

In [157]:
df_merged_one_hot = df_merged_one_hot.drop(columns=["Tipo de vino","Url"])

In [158]:
df_merged_one_hot[["Nombre","País","Región","Bodega"]] = df_merged_one_hot[["Nombre","País","Región","Bodega"]].astype(str)

In [159]:
df_merged_one_hot_droped= df_merged_one_hot.drop(columns=["Año","Uva","Nombre","Maridajes","Taste_Notes_1_1","Taste_Notes_1_2","Taste_Notes_1_3","Taste_Notes_2_1","Taste_Notes_2_2","Taste_Notes_2_3","Taste_Notes_3_1","Taste_Notes_3_2","Taste_Notes_3_3"])

In [160]:
df_merged_one_hot_droped

,ID,País,Región,Bodega,Precio,Valoración,Contenido de alcohol,Ligero/Poderoso,Seco/Dulce,Débil/Ácido,...,Riesling,Roussanne,SauvignonBlanc,Sémillon,Verdejo,Vermentino,Viognier,Viura,Weissburgunder,Xarel-lo
0,15819,Italia,Vermentino di Gallura,Sella & Mosca,16.38,3.7,5,5.8,0.4,3.4,...,0,0,0,0,0,1,0,0,0,0
1,1514997,Austria,Neusiedlersee-Hügelland,Triebaumer,13.69,3.8,13,2.0,2.1,8.0,...,0,0,0,0,0,0,0,0,0,0
2,4262558,España,Valencia,Vegalfaro,12.00,3.5,95,4.7,0.0,4.1,...,0,0,0,0,0,0,0,0,0,0
3,5707221,Italia,Montefalco,Adanti,13.84,3.5,14,4.1,1.6,5.1,...,0,0,0,0,0,0,0,0,0,0
4,7009177,Francia,Bourgogne,Domaine de Rochebin,12.90,3.9,No disponible,4.5,0.6,3.4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,1736573,Italia,Bianco di Custoza,Corte Viola,7.98,3.0,11,No disponible,No disponible,No disponible,...,0,0,0,0,0,0,0,0,0,0
11657,3125229,Francia,Côtes de Gascogne,Domaine Saint-Lannes,6.40,3.8,12,No disponible,No disponible,No disponible,...,0,0,1,0,0,0,0,0,0,0
11658,2219168,España,País Vasco,Vinos Atxega,5.00,3.1,11,No disponible,No disponible,No disponible,...,0,0,0,0,0,0,0,0,0,0
11659,5233952,Alemania,Pfalz,Bergdolt-Reif & Nett,8.50,3.8,19,No disponible,No disponible,No disponible,...,0,0,0,0,0,0,0,0,0,0


In [161]:
df_merged_one_hot_droped = df_merged_one_hot_droped[df_merged_one_hot_droped["Ligero/Poderoso"] != "No disponible"]

In [162]:
df_merged_one_hot_droped[df_merged_one_hot_droped["Ligero/Poderoso"] == "No disponible"]

,ID,País,Región,Bodega,Precio,Valoración,Contenido de alcohol,Ligero/Poderoso,Seco/Dulce,Débil/Ácido,...,Riesling,Roussanne,SauvignonBlanc,Sémillon,Verdejo,Vermentino,Viognier,Viura,Weissburgunder,Xarel-lo


In [163]:
df_merged_one_hot_droped

,ID,País,Región,Bodega,Precio,Valoración,Contenido de alcohol,Ligero/Poderoso,Seco/Dulce,Débil/Ácido,...,Riesling,Roussanne,SauvignonBlanc,Sémillon,Verdejo,Vermentino,Viognier,Viura,Weissburgunder,Xarel-lo
0,15819,Italia,Vermentino di Gallura,Sella & Mosca,16.38,3.7,5,5.8,0.4,3.4,...,0,0,0,0,0,1,0,0,0,0
1,1514997,Austria,Neusiedlersee-Hügelland,Triebaumer,13.69,3.8,13,2.0,2.1,8.0,...,0,0,0,0,0,0,0,0,0,0
2,4262558,España,Valencia,Vegalfaro,12.00,3.5,95,4.7,0.0,4.1,...,0,0,0,0,0,0,0,0,0,0
3,5707221,Italia,Montefalco,Adanti,13.84,3.5,14,4.1,1.6,5.1,...,0,0,0,0,0,0,0,0,0,0
4,7009177,Francia,Bourgogne,Domaine de Rochebin,12.90,3.9,No disponible,4.5,0.6,3.4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11050,9349980,Austria,Kremstal,Heidelinde & Markus Lang,51.48,4.2,11,1.2,0.0,6.5,...,0,0,0,0,0,0,0,0,0,0
11051,1458164,Francia,"Alsace Grand Cru ""Eichberg""",Paul Ginglinger,59.30,4.0,14,5.0,2.6,7.5,...,1,0,0,0,0,0,0,0,0,0
11052,5863795,Francia,"Chablis Grand Cru ""Bougros""",Roland Lavantureux,48.93,4.3,5,4.1,0.2,6.8,...,0,0,0,0,0,0,0,0,0,0
11053,5309011,Francia,Sancerre,Henri Bourgeois,51.00,4.1,No disponible,1.9,0.9,6.6,...,0,0,1,0,0,0,0,0,0,0


In [164]:
df_merged_one_hot_droped.to_csv("df_mergedf_blancos.csv")